In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pylab as plt
from common import gsave, gload
import pandas as pd

from imagenet_helpers import ImageNetHierarchy

In [2]:
f = np.load('resnet50.npy')[:, 0]
y = np.load('val_labels.npy')
f.shape, y.shape

((50000,), (50000,))

In [3]:
print('ImageNet accuracy:', (f == y).mean())

ImageNet accuracy: 0.7613


In [4]:
net = ImageNetHierarchy('./', './')

In [5]:
all_dog_wnlabels = []
for wnid in net.tree['n02084071'].descendants_all: ## dog
    #print(f"Subclass | WordNet ID: {wnid}, Name: {net.wnid_to_name[wnid]}")
    all_dog_wnlabels.append(wnid)
    
# https://gist.githubusercontent.com/fnielsen/4a5c94eaa6dcdf29b7a62d886f540372/raw/d25516d26be4a8d3e0aeebe9275631754b8e2c73/imagenet_label_to_wordnet_synset.txt
with open('imagenet_label_to_wordnet_synset.txt') as file:
    label_to_wordnet = eval(file.read())
    
valid_wnids = { 'n'+v['id'][:-2] for k, v in label_to_wordnet.items() } # all wordnet ids that are imagenet labels
wnid_to_label = { 'n'+v['id'][:-2] : k for k, v in  label_to_wordnet.items()  }

imagenet_dogs = { wnid_to_label[d] for d in all_dog_wnlabels if d in valid_wnids } # imagenet labels that are dogs
imagenet_dogs = list(imagenet_dogs)

## terriers
terrier_wnids = net.tree['n02092468'].descendants_all
imagenet_terriers = list({ wnid_to_label[d] for d in terrier_wnids if d in valid_wnids })

## retriever
# terrier_wnids = net.tree['n02093056'].descendants_all
# imagenet_terriers = list({ wnid_to_label[d] for d in terrier_wnids if d in valid_wnids })

In [6]:
f = np.load('alexnet.npy')[:, 0]
y = np.load('val_labels.npy')
f.shape, y.shape

L_dog = np.isin(y, imagenet_dogs)
f_dog = np.isin(f, imagenet_dogs)

L_terr = np.isin(y, imagenet_terriers)
f_terr = np.isin(f, imagenet_terriers)

In [7]:
print('Fraction of dogs:', L_dog.mean())
print('Types of dogs:', len(imagenet_dogs))
print('Types of terriers:', len(imagenet_terriers))
print('---')
print('ImageNet accuracy:', (f == y).mean())
print('> Accuracy on dogs:', (f[L_dog] == y[L_dog]).mean())
print('> Accuracy on terriers:', (f[L_terr] == y[L_terr]).mean())
print('Accuracy for binary {dog/not-dog}:', (f_dog == L_dog).mean())
print('Accuracy on {terrier/not-terrier} restricted to Dogs:', (f_terr[L_dog] == L_terr[L_dog]).mean())
print('---')
print('Fraction of real Terriers / Test-Dogs : ', L_terr[L_dog].sum() / L_dog.sum())
print('Fraction of output Terriers / Test-Dogs: ', f_terr[L_dog].sum() / L_dog.sum())

Fraction of dogs: 0.116
Types of dogs: 116
Types of terriers: 26
---
ImageNet accuracy: 0.56518
> Accuracy on dogs: 0.5882758620689655
> Accuracy on terriers: 0.5723076923076923
Accuracy for binary {dog/not-dog}: 0.98428
Accuracy on {terrier/not-terrier} restricted to Dogs: 0.9129310344827586
---
Fraction of real Terriers / Test-Dogs :  0.22413793103448276
Fraction of output Terriers / Test-Dogs:  0.20913793103448275


In [8]:
def get_rec(fname, name):
    f = np.load(fname)[:, 0]
    y = np.load('val_labels.npy')

    L_dog = np.isin(y, imagenet_dogs)
    f_dog = np.isin(f, imagenet_dogs)

    L_terr = np.isin(y, imagenet_terriers)
    f_terr = np.isin(f, imagenet_terriers)
    
    rec = {
        'Model' : name,
        'ImageNet Accuracy' : (f == y).mean(),
        'Accuracy on dogs' : (f[L_dog] == y[L_dog]).mean(),
        'Accuracy on terriers': (f[L_terr] == y[L_terr]).mean(),
        'Accuracy for binary {dog/not-dog}' : (f_dog == L_dog).mean(),
        'Accuracy on {terrier/not-terrier} among dogs': (f_terr[L_dog] == L_terr[L_dog]).mean(),
        'Fraction of real-terriers among dogs' : L_terr[L_dog].sum() / L_dog.sum(),
        'Fraction of predicted-terriers among dogs' : f_terr[L_dog].sum() / L_dog.sum()
    }
    return rec

todo = [('alexnet.npy', 'AlexNet'),
        ('resnet18.npy', 'ResNet18'),
        ('resnet50.npy', 'ResNet50'),
        ('bagnet8.npy', 'BagNet8'),
        ('bagnet32.npy', 'BagNet32')]

#get_rec(*todo[-1])

df = pd.DataFrame([get_rec(f, n) for f, n in todo])

In [9]:
df

,Model,ImageNet Accuracy,Accuracy on dogs,Accuracy on terriers,Accuracy for binary {dog/not-dog},Accuracy on {terrier/not-terrier} among dogs,Fraction of real-terriers among dogs,Fraction of predicted-terriers among dogs
0,AlexNet,0.56518,0.588276,0.572308,0.98428,0.912931,0.224138,0.209138
1,ResNet18,0.69758,0.728966,0.703846,0.99334,0.955345,0.224138,0.221897
2,ResNet50,0.76130,0.792931,0.774615,0.99600,0.968793,0.224138,0.228793
3,BagNet8,0.46386,0.461724,0.420769,0.97212,0.875517,0.224138,0.192414
4,BagNet32,0.66664,0.700517,0.659231,0.99222,0.943793,0.224138,0.215172


In [10]:
print(df.to_latex(index=False, float_format="%.3f"))

\begin{tabular}{lrrrrrrr}
\toprule
    Model &  ImageNet Accuracy &  Accuracy on dogs &  Accuracy on terriers &  Accuracy for binary \{dog/not-dog\} &  Accuracy on \{terrier/not-terrier\} among dogs &  Fraction of real-terriers among dogs &  Fraction of predicted-terriers among dogs \\
\midrule
  AlexNet &              0.565 &             0.588 &                 0.572 &                              0.984 &                                         0.913 &                                 0.224 &                                      0.209 \\
 ResNet18 &              0.698 &             0.729 &                 0.704 &                              0.993 &                                         0.955 &                                 0.224 &                                      0.222 \\
 ResNet50 &              0.761 &             0.793 &                 0.775 &                              0.996 &                                         0.969 &                                 0.224 &       

In [11]:
df.set_index('Model').T

Model,AlexNet,ResNet18,ResNet50,BagNet8,BagNet32
ImageNet Accuracy,0.565180,0.697580,0.761300,0.463860,0.666640
Accuracy on dogs,0.588276,0.728966,0.792931,0.461724,0.700517
Accuracy on terriers,0.572308,0.703846,0.774615,0.420769,0.659231
Accuracy for binary {dog/not-dog},0.984280,0.993340,0.996000,0.972120,0.992220
Accuracy on {terrier/not-terrier} among dogs,0.912931,0.955345,0.968793,0.875517,0.943793
Fraction of real-terriers among dogs,0.224138,0.224138,0.224138,0.224138,0.224138
Fraction of predicted-terriers among dogs,0.209138,0.221897,0.228793,0.192414,0.215172


In [12]:
print(df.set_index('Model').T.to_latex(index=True, float_format="%.3f"))

\begin{tabular}{lrrrrr}
\toprule
Model &  AlexNet &  ResNet18 &  ResNet50 &  BagNet8 &  BagNet32 \\
\midrule
ImageNet Accuracy                            &    0.565 &     0.698 &     0.761 &    0.464 &     0.667 \\
Accuracy on dogs                             &    0.588 &     0.729 &     0.793 &    0.462 &     0.701 \\
Accuracy on terriers                         &    0.572 &     0.704 &     0.775 &    0.421 &     0.659 \\
Accuracy for binary \{dog/not-dog\}            &    0.984 &     0.993 &     0.996 &    0.972 &     0.992 \\
Accuracy on \{terrier/not-terrier\} among dogs &    0.913 &     0.955 &     0.969 &    0.876 &     0.944 \\
Fraction of real-terriers among dogs         &    0.224 &     0.224 &     0.224 &    0.224 &     0.224 \\
Fraction of predicted-terriers among dogs    &    0.209 &     0.222 &     0.229 &    0.192 &     0.215 \\
\bottomrule
\end{tabular}



In [19]:
print(df.set_index('Model').T[['AlexNet', 'ResNet50']].to_latex(index=True, float_format="%.3f"))

\begin{tabular}{lrr}
\toprule
Model &  AlexNet &  ResNet50 \\
\midrule
ImageNet Accuracy                            &    0.565 &     0.761 \\
Accuracy on dogs                             &    0.588 &     0.793 \\
Accuracy on terriers                         &    0.572 &     0.775 \\
Accuracy for binary \{dog/not-dog\}            &    0.984 &     0.996 \\
Accuracy on \{terrier/not-terrier\} among dogs &    0.913 &     0.969 \\
Fraction of real-terriers among dogs         &    0.224 &     0.224 \\
Fraction of predicted-terriers among dogs    &    0.209 &     0.229 \\
\bottomrule
\end{tabular}

